In [ ]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.metrics import log_loss

from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn import svm


In [ ]:
df = pd.read_csv("../MachinehackGlassQualityPrediction/Glass_Quality_Participants_Data/Train.csv")
#tf = pd.read_csv("../MachinehackGlassQualityPrediction/Glass_Quality_Participants_Data/Test.csv")

#sf = pd.read_excel("../MachinehackGlassQualityPrediction/Glass_Quality_Participants_Data/Sample_Submission.xlsx")


In [ ]:
df['xmax'] = np.sqrt(df['xmax'])
df['ymax'] = np.sqrt(df['ymax'])
df['xmin'] = np.sqrt(df['xmin'])
df['ymin'] = np.sqrt(df['ymin'])

df['max_luminosity'] = np.log(df['max_luminosity'])
df['pixel_area'] = np.sqrt(df['pixel_area'])
df['log_area'] = np.sqrt(df['log_area'])

# tf['xmax'] = np.sqrt(tf['xmax'])
# tf['ymax'] = np.sqrt(tf['ymax'])
# tf['xmin'] = np.sqrt(tf['xmin'])
# tf['ymin'] = np.sqrt(tf['ymin'])

# tf['max_luminosity'] = np.log(tf['max_luminosity'])
# tf['pixel_area'] = np.sqrt(tf['pixel_area'])
# tf['log_area'] = np.sqrt(tf['log_area'])



In [ ]:
#df =df.drop('pixel_area',axis=1)

# tf =tf.drop('log_area',axis=1)

In [ ]:
# adding  interaction new features 
import itertools
interactions = pd.DataFrame(index=df.index)
interactions

In [ ]:
features =['max_luminosity','thickness','pixel_area']
for col1 ,col2 in  itertools.combinations(features,2):
    
    newcolname = col1 + "_" + col2 
    new_values = df[col1].map(str) + "_" + df[col2].map(str)
    interactions[newcolname] = new_values


In [ ]:
# interactions['max_luminosity_thickness']

In [ ]:
#df =df.join(interactions)

In [ ]:
# interactionstest = pd.DataFrame(index=tf.index)
# interactionstest

In [ ]:
# features =['max_luminosity','thickness','pixel_area']
# for col1 ,col2 in  itertools.combinations(features,2):
    
#     newcolname = col1 + "_" + col2 
#     new_values = tf[col1].map(str) + "_" + tf[col2].map(str)
#     interactionstest[newcolname] = new_values


In [ ]:
# tf =tf.join(interactionstest)

In [ ]:
# tf
# df['class'] = df['class'].replace(2,0)

In [ ]:
df.head()

In [ ]:
y = df['class']

# To keep things simple, we'll use only numerical predictors
X = df.drop(['class'], axis=1)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.70, test_size=0.30,random_state=0)

In [ ]:
enc = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='kmeans')
enc.fit(np.array(X_train['max_luminosity']).reshape(-1,1))
X_train['max_luminosity_binned'] = enc.transform(np.array(X_train['max_luminosity']).reshape(-1,1))
X_valid['max_luminosity_binned'] = enc.transform(np.array(X_valid['max_luminosity']).reshape(-1,1))


In [ ]:
enc = KBinsDiscretizer(n_bins=7, encode='ordinal', strategy='kmeans')
enc.fit(np.array(X_train['thickness']).reshape(-1,1))
X_train['thickness_binned'] = enc.transform(np.array(X_train['thickness']).reshape(-1,1))
X_valid['thickness_binned'] = enc.transform(np.array(X_valid['thickness']).reshape(-1,1))


In [ ]:
X_train.head()

In [ ]:

def targetencoding(train,test,y_train):
    import category_encoders as ce
    # Create the encoder itself
    cat_features = ['max_luminosity_thickness','max_luminosity_pixel_area','thickness_pixel_area']
    print(f'targest emcoding for features {cat_features}')
    target_enc = ce.TargetEncoder(cols=cat_features)

    

    # Fit the encoder using the categorical features and target
    target_enc.fit(train[cat_features], y_train)
    

    # Transform the features, rename the columns with _target suffix, and join to dataframe
    traintrgtenc = train.join(target_enc.transform(train[cat_features]).add_suffix('_target'))
    testtrgtenc = test.join(target_enc.transform(test[cat_features]).add_suffix('_target'))

    traintrgtenc = traintrgtenc.drop(cat_features, axis =1)
    testtrgtenc = testtrgtenc.drop(cat_features, axis =1)

    
    print(traintrgtenc.shape,testtrgtenc.shape)
    
    return traintrgtenc,testtrgtenc

In [ ]:
#X_traint,X_validt = targetencoding(X_train,X_valid,y_train)

In [ ]:
X_train.head()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
    
# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    
    
    clf1 = svm.SVC(kernel='linear', random_state=2,probability=True)
    clf1.fit(X_train,y_train)
    
    
    preds = clf1.predict(X_valid)
    predsforlog = clf1.predict_proba(X_valid)
    
    
    target_names = ['class_0','class_1']
    print(classification_report(y_valid, preds, target_names=target_names,labels= [1,2]))
    
    print(confusion_matrix(y_valid,preds))
    return log_loss(y_valid,predsforlog) 

In [ ]:
X_train.dtypes

In [ ]:
n_folds=5
def logloss(model):
    kf = KFold(n_folds, shuffle=True, random_state=2).get_n_splits(X_train.values)
    logloss= (-cross_val_score(model, X_train.values, y_train, scoring="neg_log_loss", cv = kf))
    return(logloss)

In [ ]:
 score_dataset(X_train,X_valid,y_train,y_valid)

In [ ]:
l = logloss(
        svm.SVC(random_state=2,probability=True))
    
print("(avg {0} ,std{1})".format(l.mean(),l.std()))
